In [ ]:
# cifar10 playground with pytorch

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.backends.cudnn as cudnn
from torch.autograd import Variable

import torchvision 
import torchvision.transforms as transforms

import matplotlib.pyplot as plt


import os
import argparse

import import_ipynb
from VGGNet import *
from GoogLeNet import *
from pResNet import *
from utils import *


In [ ]:
parser = argparse.ArgumentParser(description='CIFAR-10 playground with pytorch')
parser.add_argument('--lr', default=0.1, type=float, help='learning rate')
parser.add_argument('--resume', '-r', action = 'store_true', help = 'training resume from chekpoint')
parser.add_argument('--optim', '-o', )
parser.add_argument('--loss', '-l',)
parser.add_argument('--net', '-n',)
parser.add_argument('--epoch','-e')
parser.add_argument('--batchsize','-b')
args = parser.parse_args(args=[])

use_cuda = torch.cuda.is_available()
best_acc = 0
start_epoch = 0

# Data preprocessing
print('Preparing data..')
train_trans = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465),(0.2023, 0.1994, 0.2010)),
])

test_trans = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465),(0.2023, 0.1994, 0.2010)),
])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                       download=True, transform=train_trans)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64,
                                         shuffle=False, num_workers=4)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                      download=True, transform=test_trans)
testloader = torch.utils.data.DataLoader(testset, batch_size=64,
                                        shuffle=False, num_workers=4)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

# Chose model
if args.resume:
    print('Resuming from checkpoint')
    assert os.path.isdir('chechkpoint'),'Error: no checkpoint directory found'
    checkpoint = torch.load('./checkpoint/ckpt.pkl')
    net = checkpoint['net']
    best_acc = checkpoint['acc']
    start_epoch = checkpoint['epoch']
else:
    print('Building a model')
#     net = VGG('VGG19')
#     net = GoogLeNet()
    net = pResNet18()
    
    
if use_cuda:
    net.cuda()
    net = torch.nn.DataParallel(net, device_ids=range(torch.cuda.device_count()))
    cudnn.benchmark = True
    
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=args.lr, momentum=0.9,weight_decay=5e-4)


# Training process
def train(epoch):
    print('\nEpoch:%d'%epoch)
    net.train()
    train_loss = 0
    correct = 0
    total = 0
    for i, (inputs, labels) in enumerate(trainloader):
        # trainig wether use cuda or not
        if use_cuda:
            inputs, labels = inputs.cuda(), labels.cuda()
        # transer data to autograd type
        inputs, labels = Variable(inputs), Variable(labels)
        # zero the parameters gradients
        optimizer.zero_grad()
        # forward + backward + optimize
        outputs = net(inputs)
#         print(outputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        train_loss += loss.data[0]
        # chose the max probability class as predicted
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += predicted.eq(labels.data).cpu().sum()
#         print(len(trainloader))
#         print(labels.size(0))
        progressBar(i, len(trainloader), 'Loss: {0:0.3}|Acc:{1:0.3}% {2}/{3}'
            .format(train_loss/(i+1), 100.*correct/total, correct, total))
        
def test(epoch):
    global best_acc
    net.eval()
    test_loss = 0
    correct = 0
    total = 0
    for i , (inputs, labels) in enumerate(testloader):
        if use_cuda:
            inputs, labels = inputs.cuda(), labels.cuda()
        inputs, labels = Variable(inputs, volatile=True), Variable(labels)
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        
        test_loss += loss.data[0]
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += predicted.eq(labels.data).cpu().sum()
        
        progressBar(i, len(testloader), 'Loss: %.3f|Acc: %.3f%% (%d/%d)'
            % (test_loss/(i+1), 100.*correct/total, correct, total))
        
        # Save checkpoint
        
        acc = 100.* correct / total
        if acc > best_acc:
            print('Saving model..')
            state = {
                'net': net.module if use_cuda else net,
                'acc': acc,
                'epoch': epoch,
            }
            if not os.path.isdir('checkpoint'):
                os.mkdir('checkpoint')
            torch.save(state, './checkpoint/ckpt.pth')
            best_acc = acc

# control lr until fix epoch
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.1)
for epoch in range(start_epoch, start_epoch+50):
    scheduler.step()
    train(epoch)
    if epoch // 50 == 0:
        test(epoch)
    